In [1]:
# local에 padnas, numpy, matplotlib 설치필요
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from pyspark.sql.types import StructField, StructType, StringType, DoubleType, FloatType, IntegerType
from pyspark.sql.functions import col

# local에 pyspark 설치필요
!pip install pyspark
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

# # Spark 세션 생성
spark = SparkSession.builder \
    .appName("RS_AmazonXTwitter") \
    .getOrCreate()

In [2]:
import gdown

# 파일 ID와 URL 생성
# file_id = "1E3eGva2ZaSTLltOhQ-SDOB4h6dVEM-4E"
# tweet_url = f"https://drive.google.com/uc?id={file_id}"
# output_file = "tweet.csv"  # 저장할 파일 이름
# gdown.download(tweet_url, output_file, quiet=False)

# 파일 ID와 URL 생성
# 공통 상품 코드만 남긴 데이터
file_id = "1GpYNkh6rs0EY9aMQIFvOoVJUEFaKPfMr"
fashion_review_url = f"https://drive.google.com/uc?id={file_id}"
output_file = "common_fashion_review.jsonl"  # 저장할 파일 이름
gdown.download(fashion_review_url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1GpYNkh6rs0EY9aMQIFvOoVJUEFaKPfMr
To: /content/common_fashion_review.jsonl
100%|██████████| 9.25M/9.25M [00:00<00:00, 35.5MB/s]


'common_fashion_review.jsonl'

# 1. Twitter data

In [3]:
# # file path
# input_file = "/content/tweet.csv"

# # raw data
# tw_raw_df = spark.read.format("csv").option("header", "false").load(input_file)

# # set schema
# tw_schema = StructType([
#     StructField("target", DoubleType(), True),   # Sentiment (0/1)
#     StructField("id", DoubleType(), True),       # Tweet ID
#     StructField("date", StringType(), True),     # Date
#     StructField("query", StringType(), True),    # Query (if applicable)
#     StructField("author", StringType(), True),   # Author/User
#     StructField("content", StringType(), True),  # Tweet content
# ])

# tw_formatted_df = tw_raw_df.toDF(*[field.name for field in tw_schema.fields])

# # select needed column
# tw_formatted_df = tw_formatted_df.select("id", "author", "content")

# # check data
# tw_formatted_df.printSchema()
# tw_formatted_df.show(5)

# authors = tw_formatted_df.select("author").distinct()

# print("total count : ",tw_formatted_df.count())
# print("authors count : ",authors.count())

**1.0. Preprocessing**

**1-1. Categorized tweet by dictionary**

In [4]:
# # from pyspark.sql.functions import col, lower, when, size, array, expr
# from pyspark.sql.functions import explode, split, col, lower, count, desc, max

# # 입력 파일 경로
# input_file = "/content/training.1600000.processed.noemoticon.csv"

# # 데이터 스키마 정의
# tw_schema = StructType([
#     StructField("target", DoubleType(), True),
#     StructField("id", DoubleType(), True),
#     StructField("date", StringType(), True),
#     StructField("query", StringType(), True),
#     StructField("author", StringType(), True),
#     StructField("content", StringType(), True)
# ])

# # 데이터 로드 및 스키마 적용
# tw_raw_df = spark.read.format("csv").option("header", "false").load(input_file)
# tw_formatted_df = tw_raw_df.toDF(*[field.name for field in tw_schema.fields])

# # 데이터 샘플링
# categorized_df = tw_formatted_df.select("content").sample(fraction=0.01, seed=42)

# # 카테고리 키워드 딕셔너리 정의
# category_keywords = {
#     "jewelry": ["ring", "necklace", "bracelet", "earrings", "pendant", "brooch", "chain", "charm", "gold", "silver"],
#     "watch": ["watch", "timepiece", "chronograph", "quartz", "analog", "digital", "wristwatch", "strap", "dial", "bezel"],
#     "drugstore": ["pharmacy", "medicine", "drugs", "prescription", "pill", "vitamin", "tablet", "pharmacist", "aspirin", "capsule"],
#     "pet_products": ["pet", "dog", "cat", "fish", "bird", "hamster", "leash", "kennel", "collar", "aquarium"],
#     "automotive": ["car", "vehicle", "engine", "automobile", "tire", "wheel", "battery", "transmission", "brake", "sedan"],
#     "shoes": ["shoe", "sneakers", "boots", "heels", "sandals", "loafers", "oxfords", "slippers", "cleats", "flats"],
#     "beauty": ["beauty", "makeup", "cosmetics", "lipstick", "skincare", "mascara", "foundation", "blush", "palette", "concealer"],
#     "office_product": ["office", "stationery", "desk", "printer", "paper", "pen", "notebook", "folder", "calculator", "stapler"],
#     "digital_ebook_purchase": ["ebook", "kindle", "digital book", "pdf", "epub", "audiobook", "download", "reader", "publication", "novel"],
#     "lawn_and_garden": ["garden", "lawn", "yard", "mower", "fertilizer", "plant", "seeds", "shovel", "hedge", "sprinkler"],
#     "kitchen": ["kitchen", "cookware", "appliance", "utensils", "cutlery", "oven", "stove", "pan", "mixer", "microwave"],
#     "apparel": ["clothing", "apparel", "dress", "shirt", "jacket", "jeans", "skirt", "sweater", "suit", "blouse"],
#     "industrial_supplies": ["industrial", "tools", "hardware", "equipment", "machinery", "fastener", "drill", "screwdriver", "wrench", "saw"],
#     "wireless": ["wireless", "network", "wifi", "router", "modem", "signal", "antenna", "hotspot", "connectivity", "bluetooth"],
#     "digital_video_download": ["video", "movie", "streaming", "download", "hd", "4k", "bluray", "rental", "cinema", "series"],
#     "grocery": ["food", "grocery", "snacks", "vegetables", "fruits", "bread", "milk", "meat", "cereal", "beverage"],
#     "personal_care_appliances": ["care", "appliance", "shaver", "toothbrush", "trimmer", "hairdryer", "curler", "straightener", "massager", "razor"],
#     "furniture": ["furniture", "sofa", "table", "chair", "bed", "dresser", "cabinet", "shelf", "couch", "armchair"],
#     "baby_product": ["baby", "infant", "crib", "diaper", "stroller", "bottle", "pacifier", "highchair", "blanket", "bib"],
#     "electronics": ["electronics", "device", "gadget", "charger", "headphones", "camera", "tablet", "speaker", "remote", "monitor"],
#     "sports": ["sports", "equipment", "fitness", "workout", "gear", "tennis", "soccer", "basketball", "yoga", "cycling"],
#     "home": ["home", "house", "living", "decor", "curtains", "lighting", "rug", "pillow", "mirror", "wall art"],
#     "pc": ["pc", "laptop", "computer", "keyboard", "mouse", "monitor", "cpu", "notebook", "desktop", "hardware"],
#     "luggage": ["luggage", "bag", "suitcase", "backpack", "duffle", "trolley", "pouch", "wallet", "briefcase", "travel"],
#     "home_improvement": ["improvement", "renovation", "diy", "paint", "tiles", "fixtures", "drill", "ladder", "flooring", "cement"],
#     "toy": ["toy", "game", "doll", "lego", "puzzle", "blocks", "teddy", "action figure", "boardgame", "train"],
#     "camera": ["camera", "dslr", "photo", "lens", "tripod", "shutter", "focus", "mirrorless", "film", "flash"],
#     "book": ["book", "novel", "read", "fiction", "non-fiction", "magazine", "literature", "paperback", "hardcover", "author"],
#     "video_games": ["game", "console", "xbox", "playstation", "nintendo", "pc gaming", "joystick", "controller", "esports", "arcade"],
#     "musical_instruments": ["instrument", "guitar", "piano", "violin", "drums", "flute", "keyboard", "trumpet", "saxophone", "harp"]
# }


# # 카테고리 키워드 데이터프레임 생성
# categories = [(category, keyword) for category, keywords in category_keywords.items() for keyword in keywords]
# category_df = spark.createDataFrame(categories, ["category", "keyword"])

# # 트윗 내용을 단어로 분리
# words_df = categorized_df.withColumn("word", explode(split(lower(col("content")), "\\W+")))

# # 키워드와 트윗 단어 매칭
# matched_df = words_df.join(
#     category_df,
#     words_df.word == category_df.keyword,
#     "inner"
# ).groupBy("content", "category").agg(count("keyword").alias("match_count"))

# categorized_df_with_alias = categorized_df.alias("categorized")
# matched_df_with_alias = matched_df.alias("matched")

# all_matched_df = categorized_df_with_alias.join(
#     matched_df_with_alias,
#     col("categorized.content") == col("matched.content"),
#     "left_outer"
# ).select(
#     col("categorized.content").alias("content"),  # categorized_df의 content 열 선택
#     coalesce(col("matched.category"), lit("others")).alias("category"),  # 매칭되지 않은 경우 'others'로 설정
#     coalesce(col("matched.match_count"), lit(0)).alias("match_count")   # 매칭되지 않은 경우 0으로 설정
# )

# # 최종 데이터 출력
# all_matched_df.show(truncate=False)

In [5]:
# # 1. 카테고리별 개수 집계
# category_counts = all_matched_df.groupBy("category").count()

# # 2. 전체 데이터 개수 계산
# total_count = all_matched_df.count()

# # 3. 비율 계산 (새로운 컬럼 추가)
# category_ratios = category_counts.withColumn(
#     "ratio", (col("count") / total_count) * 100
# )

# # 4. Spark DataFrame을 Pandas DataFrame으로 변환
# category_ratios_pd = category_ratios.toPandas()

# # 5. 시각화: 눈금 조정
# plt.figure(figsize=(10, 6))
# plt.bar(category_ratios_pd['category'], category_ratios_pd['ratio'], alpha=0.7)
# plt.title('Category Ratio Distribution', fontsize=16)
# plt.xlabel('Category', fontsize=14)
# plt.ylabel('Ratio (%)', fontsize=14)

# # Y축 눈금과 범위 설정 (예: 최대값을 50%로 제한)
# plt.ylim(0, 50)  # Y축 범위를 0~50으로 설정
# plt.yticks(range(0, 51, 10))  # 0부터 50까지 10 단위 눈금

# # X축 라벨 설정
# plt.xticks(rotation=45)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()

# # 6. 그래프 출력
# plt.show()

In [6]:
# specific_category = "automotive"
# filtered_tweets = all_matched_df.filter(col("category") == specific_category)
# filtered_tweets.show(truncate=False)

**1-2. Categorized tweet by TF-IDF**

# 2. Amazon Data

**2-1. Used Data**

*   https://amazon-reviews-2023.github.io/






# Fashion Review 데이터

# Colaborative Filtering


In [7]:
def recommend_collab(product_asin):
    # 상품 이름과 인덱스 매핑
    if product_asin not in pivot_pandas.index:
        print(f"Product '{product_asin}' not found in pivot table!")
        return

    # 특정 상품의 인덱스 찾기
    index = np.where(pivot_pandas.index == product_asin)[0][0]

    # 해당 상품의 유사도 가져오기
    similar_items = sorted(
        list(enumerate(similarity_score[index])),
        key=lambda x: x[1],
        reverse=True
    )[1:4]  # 상위 3개의 유사 상품 선택 (자기 자신 제외)

    # 유사 상품 이름 출력
    print(f"Recommendations for '{product_asin}':")
    for i in similar_items:
        similar_product = pivot_pandas.index[i[0]]
        similarity = i[1]
        print(f"Product: {similar_product}, Similarity: {similarity:.4f}")

In [8]:
common_fashion_review_path = "/content/common_fashion_review.jsonl"

common_fashion_review_df = spark.read.json(common_fashion_review_path)
common_fashion_review_df.printSchema()

common_filtered_review_df = common_fashion_review_df.select("asin","rating","user_id")

common_filtered_review_df = common_filtered_review_df \
    .withColumn("rating", col("rating").cast(FloatType()))\

review_schema = StructType([
    StructField("_asin", StringType(), False),
    StructField("rating", FloatType(), False),
    StructField("user_id", StringType(), False),
])

formatted_common_fashion_review_df = spark.createDataFrame(common_filtered_review_df.rdd, schema=review_schema)

formatted_common_fashion_review_df.printSchema()

# '_asin' 컬럼에서 고유한 값만 추출
unique_asins_df = formatted_common_fashion_review_df.select("_asin").distinct()

# 결과 확인
unique_asins_df.show()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)

root
 |-- _asin: string (nullable = false)
 |-- rating: float (nullable = false)
 |-- user_id: string (nullable = false)

+----------+
|     _asin|
+----------+
|B07CZT1ZHH|
|B084BSDBLG|
|B0882PV16R|
|B0B96SNMPL|
|B0BBT6W6CW|
|B017C52C7Y|
|B08KSDRN9Q|
|B09XFXF3JS|
|B07BQ7XFCW|
|B07XSNCFYF|
|B09F9N2ZYT|
|B00J1POO8E|
|B0

In [9]:
from pyspark.sql.functions import col, count

# Step 1: _asin과 user_id 조합의 중복 제거
distinct_review_df = formatted_common_fashion_review_df.dropDuplicates(["user_id", "_asin"])

# Step 2: user_id 빈도 계산
user_id_count_df = distinct_review_df \
    .groupBy("user_id") \
    .agg(count("user_id").alias("user_id_count"))

# Step 3: 중복 제거된 데이터와 user_id_count 병합
sorted_review_df = distinct_review_df \
    .join(user_id_count_df, on="user_id") \
    .orderBy(col("user_id_count").desc(), col("user_id").asc()) # user_id_count 내림차순, user_id 오름차순

# Step 4: 정렬된 데이터에서 user_id_count 제거 (필요하면)
sorted_review_df = sorted_review_df.drop("user_id_count")

# 결과 확인
sorted_review_df.show()


+--------------------+----------+------+
|             user_id|     _asin|rating|
+--------------------+----------+------+
|AE7ZJ5A3CAMZJ6JF6...|B07WGQKNHP|   2.0|
|AE7ZJ5A3CAMZJ6JF6...|B01431E52K|   5.0|
|AECHES7YQXQUNNGWI...|B09YCTKJF6|   5.0|
|AECHES7YQXQUNNGWI...|B0BF4LVF5X|   5.0|
|AEDP2SF5CUL3OHKPZ...|B098D3TSYP|   5.0|
|AEDP2SF5CUL3OHKPZ...|B000GAWSDG|   5.0|
|AEJNYNLJ2YMIBGNA7...|B087WBBS85|   5.0|
|AEJNYNLJ2YMIBGNA7...|B000GAWSDG|   5.0|
|AEKZZYS4QRSMASWKF...|B00AC6V0VA|   5.0|
|AEKZZYS4QRSMASWKF...|B00ES8KVT0|   5.0|
|AERHEGTGLSDB4KRKN...|B08KSDRN9Q|   2.0|
|AERHEGTGLSDB4KRKN...|B07P7M18C6|   2.0|
|AEVNVUMNKROOD4PUT...|B09V3B6NH6|   3.0|
|AEVNVUMNKROOD4PUT...|B000GAWSDG|   5.0|
|AF4SOQYKKASVPYCS3...|B07JLXK3B8|   5.0|
|AF4SOQYKKASVPYCS3...|B000GAWSDG|   5.0|
|AF645HKHKCO626U3M...|B07LBM5LCF|   4.0|
|AF645HKHKCO626U3M...|B000GAWSDG|   5.0|
|AF7UKMPI2AQPW3XPW...|B095H5BZ65|   2.0|
|AF7UKMPI2AQPW3XPW...|B00J4JKPBC|   5.0|
+--------------------+----------+------+
only showing top

In [11]:
sampled_df = sorted_review_df.limit(10000)
pivot_df = sampled_df.groupBy("_asin").pivot("user_id").agg({"rating": "first"})
pivot_df = pivot_df.fillna(0)
# 결과 출력

from sklearn.metrics.pairwise import cosine_similarity

# PySpark DataFrame -> Pandas DataFrame 변환
pivot_pandas = pivot_df.toPandas()

# 'asin' 컬럼 제거 (필요한 경우)
pivot_pandas = pivot_pandas.set_index("_asin")

# Null 값을 0으로 대체 (cosine_similarity는 NaN 값을 허용하지 않음)
pivot_pandas = pivot_pandas.fillna(0)

# Pandas DataFrame -> NumPy 배열 변환
pivot_numpy = pivot_pandas.values

# 코사인 유사도 계산
similarity_score = cosine_similarity(pivot_numpy)

# 결과 출력
# print(similarity_score)

In [12]:
print(similarity_score)

[[0.99999994 0.01000039 0.         ... 0.         0.         0.        ]
 [0.01000039 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.0000001  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [15]:
recommend_collab("B000GAWSDG")

Recommendations for 'B000GAWSDG':
Product: B07JLXK3B8, Similarity: 0.0217
Product: B098D3TSYP, Similarity: 0.0060
Product: B07G5JXYX3, Similarity: 0.0057
